In [1]:
#!pip install nlgmetricverse jsonlines
import json, jsonlines
from nlgmetricverse import NLGMetricverse, load_metric
from sklearn.metrics import f1_score

ESCONV_STRATEGY = [
    "[Question]",
    "[Restatement or Paraphrasing]",
    "[Reflection of feelings]",
    "[Self-disclosure]",
    "[Affirmation and Reassurance]",
    "[Providing Suggestions]",
    "[Information]",
    "[Others]"
]

/hdd1/ashok/.conda/envs/straashok/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
results = list(jsonlines.open("Ashokajou51_esconv-sorted-incontext-tinyllama-plm_epoch-2.json"))

In [3]:
acc = 0
error = 0

preds=[]
gts=[]

def get_strategy(x):
  y = x.split("]", 1)[0] + "]"
  if y in ESCONV_STRATEGY:
    return y, ESCONV_STRATEGY.index(y)
  else:
    return "Error", -1

for r in results:
  prediction = r["prediction"]
  response = r["context"][-1]["content"]

  (pred_strat, pred_strat_id), (gt_strat, gt_strat_id) = get_strategy(prediction), get_strategy(response)
  if gt_strat_id >= 0 and pred_strat == gt_strat:
    acc += 1

  preds.append(pred_strat_id)
  gts.append(gt_strat_id)

print(f"acc: {acc / len(results)}")
print(f"error: {error / len(results)}")
print("macro F1", f1_score(preds, gts, average="macro"))

predictions = [x['prediction'].split(']', 1)[1].replace("</s>", "") for x in results]
references = [response for x in results]


metrics = [
    load_metric("bleu", resulting_name="bleu_1", compute_kwargs={"max_order": 1}),
    load_metric("bleu", resulting_name="bleu_2", compute_kwargs={"max_order": 2}),
    load_metric("rouge"),
    load_metric("f1"),
    ]
scorer = NLGMetricverse(metrics=metrics)
scores = scorer(predictions=predictions, references=references, reduce_fn="mean")
print(json.dumps(scores, indent=4))

acc: 0.27505827505827507
error: 0.0
macro F1 0.171417919594705
{
    "total_items": 429,
    "empty_items": 0,
    "total_time_elapsed": 0.32628560066223145,
    "bleu_1": {
        "score": 0.0978794759604615,
        "precisions": [
            0.23271710424110612
        ],
        "brevity_penalty": 0.4205942501718896,
        "length_ratio": 0.5358807858807859,
        "translation_length": 6437,
        "reference_length": 12012,
        "time_elapsed": 0.028853178024291992
    },
    "bleu_2": {
        "score": 0.022669563247990125,
        "precisions": [
            0.23271710424110612,
            0.012483355525965379
        ],
        "brevity_penalty": 0.4205942501718896,
        "length_ratio": 0.5358807858807859,
        "translation_length": 6437,
        "reference_length": 12012,
        "time_elapsed": 0.028918981552124023
    },
    "rouge": {
        "rouge1": 0.15713282550957855,
        "rouge2": 0.007576935885583864,
        "rougeL": 0.1090242706026532,
      

In [4]:
predictions[0]

' Hello, how are you doing today?'